This notebook can be used to generate the main graphs in our paper on mechanotransduction signaling in cells on nanopillars.

First, we introduce the necessary imports and configure our matplotlib settings.

In [12]:
import sys, os, pathlib
sys.path.append("/root/shared/gitrepos/smart-nanopillars/utils")
sys.path.append("/root/shared/gitrepos/smart-comp-sci/mechanotransduction-example")
import smart_analysis
from mechanotransduction_ode import mechanotransduction_ode_calc
from matplotlib import pyplot as plt
import matplotlib
plt.style.use(str(pathlib.Path.cwd() / ".." / "utils" / "smart_plots.mplstyle"))
matplotlib.rcParams.update({'figure.figsize': (5,4)})
import numpy as np
cur_dir = str(pathlib.Path.cwd() / "..")

This runs data analysis on all simulations in all subfolders within the folder specified by `files_dir` and saves into `npy_dir`. The correct mesh must be included in a subfolder of `mesh_dir`.

In [ ]:
import re
mesh_dir = "/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars_movenuc"
files_dir = "/root/scratch/nanopillar-sims/results_nanopillars_movenuc"
npy_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "npy-files-nanopillars-movenuc"
npy_dir.mkdir(exist_ok=True)
test_folders = os.listdir(files_dir)
condition_str = []
for i in range(0,len(test_folders)):
    mesh_file = ""
    for folder in os.listdir(mesh_dir):
        if folder in test_folders[i]: # mesh name matches current case
            mesh_file = f"{mesh_dir}/{folder}/spreadCell_mesh.h5"
            break
    if mesh_file == "":
        Warning("Mesh could not be found, skipping to next case")
        condition_str.append("")
        continue
    results_folder = f"{files_dir}/{test_folders[i]}"
    condition_cur = test_folders[i]
    condition_str.append(condition_cur)
    if mesh_file=="" or results_folder=="":
        ValueError("Folders do not match expected structure for analysis")
    height = 3.0
    zmax_nuc = 6.4
    if "loc" in test_folders[i]:
        loc = float(re.findall(r"loc.*?_", test_folders[i])[0][3:-1])
    else:
        loc = 0
    tests = ["all"]#,"center_low","center_high"] # other spatial domains could be specified here
    center_low = [loc-1, -1, 0, loc+1, 1, height+1]
    center_high = [loc-1, -1, zmax_nuc-1.0, loc+1, 1, zmax_nuc+10]
    domains = [[], center_low, center_high]
    for sd in range(len(tests)):
        try:
            tVec, results_all = smart_analysis.analyze_all(
                mesh_file=mesh_file, results_path=results_folder, display=False, axisymm=False,
                subdomain=domains[sd])
        except:
            Warning("error in analysis, skipping to next case")
            continue
        results_all.insert(0, tVec) # add time as first element in list
        max_length = len(tVec)
        for j in range(len(results_all)):
            if len(results_all[j]) > max_length:
                max_length = len(results_all[j])
        for j in range(len(results_all)):
            num_zeros = max_length - len(results_all[j])
            for k in range(num_zeros):
                results_all[j].append(0)
        np.save(npy_dir / f"{condition_cur}_results_{tests[sd]}.npy", results_all)

Generate graphs of YAP/TAZ N/C over time for cells on different nanopillar substrates (Fig 2D) and store relevant steady-state concentrations.

In [ ]:
from smart import mesh_tools
import dolfin as d
radiusArray= [
 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
 0.5, 0.25, 0.5, 0.25,
 0.0, 0.0, 0.0, 0.0, 0.0]
pitchArray= [
 5.0, 2.5, 1.0, 5.0, 2.5, 1.0,
 5.0, 2.5, 5.0, 2.5,
 0.0, 0.0, 0.0, 0.0, 0.0]
heightArray=[
 1.0, 1.0, 1.0, 3.0, 3.0, 3.0,
 1.0, 1.0, 3.0, 3.0,
 0.0, 0.0, 0.0, 0.0, 0.0]
cellRadArray=[
 20.25, 18.52, 16.55, 19.93, 18.04, 15.39,
 20.01, 17.45, 18.06, 17.64,
 22.48, 18.08, 15.39, 14.18, 12.33]
EModArray=[
 10000000, 10000000, 10000000, 10000000, 10000000, 10000000, 
 10000000, 10000000, 10000000, 10000000,
 10000000, 14, 7, 3, 1]
tests_idx = [1,7,10]
yapData = [
 2.697152, 2.223328, 2.084372, 2.669816, 2.360008, 2.079816,
 2.55696, 2.264736, 2.456508, 2.365188,
 3.320408, 2.285056, 1.806156, 1.555306, 1.43672
]
yapSEMData = [
    0.100232, 0.063784, 0.054732, 0.09112, 0.072896, 0.054732,
    0.073056, 0.036528, 0.063924, 0.063924,
    0.04561, 0.04561, 0.036488, 0.031922, 0.041044
]
NVals = [
    38, 89, 59, 59, 69, 60,
    48, 58, 58, 58,
    396, 134, 164, 86, 23
]
yapStdData = np.array(yapSEMData) * np.sqrt(np.array(NVals))
curv0Array=[0, 1, 2, 5, 10]
var_names_saved = ["Cofilin_NP", "FActin", "GActin", "LaminA", "LIMK_A", "mDia_A", 
                   "Myo_A", "NPC_A", "pFAK", "RhoA_GDP", "RhoA_GTP", "ROCK_A", "Substrate_A", 
                   "YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_phos"]
plot_names = ["YAPTAZ_phos", "YAPTAZ", "YAPTAZ_nuc"]
results_idx = []
for name in plot_names:
    results_idx.append(var_names_saved.index(name) + 1) # add one because time is first el
pFAK_idx = var_names_saved.index("pFAK") + 1
RhoA_idx = var_names_saved.index("ROCK_A") + 1
FActin_idx = var_names_saved.index("FActin") + 1
Myo_idx = var_names_saved.index("Myo_A") + 1
mesh_dir = f"{cur_dir}/meshes/nanopillars_baseline"

errors_list = []
wasp_vals = [0.001, 0.01]

for wasp_val in wasp_vals:
    YAPTAZ_ratios = np.zeros([len(cellRadArray), len(curv0Array)])
    pFAK_SS = np.zeros([len(cellRadArray), len(curv0Array)])
    RhoA_GTP_SS = np.zeros([len(cellRadArray), len(curv0Array)])
    FActin_SS = np.zeros([len(cellRadArray), len(curv0Array)])
    MyoA_SS = np.zeros([len(cellRadArray), len(curv0Array)])
    errors = np.zeros([len(cellRadArray), len(curv0Array)])
    for i in [0, 1, 2, 6, 7, 10, 11, 12, 13, 14]:
        loaded_cur = mesh_tools.load_mesh(pathlib.Path(
            f"{mesh_dir}/nanopillars_h{heightArray[i]}_p{pitchArray[i]}_r{radiusArray[i]}_" 
            f"cellRad{cellRadArray[i]}") / "spreadCell_mesh.h5")
        PM_area = d.assemble(1.0*d.Measure("dx", d.create_meshview(loaded_cur.mf_facet, 10)))
        cyto_vol = d.assemble(1.0*d.Measure("dx", d.create_meshview(loaded_cur.mf_cell, 1)))
        if i >= 10:
            file_cur = (f"{cur_dir}/analysis_data/npy-files-nanopillars-withwasp/"
                    f"nanopillars_h{heightArray[i]}_p{pitchArray[i]}_r{radiusArray[i]}_" 
                    f"cellRad{cellRadArray[i]}_wasp{wasp_val}_results_all.npy")
            results_cur = np.load(file_cur)
            YAPphos = results_cur[results_idx[0]]
            YAPnp = results_cur[results_idx[1]]
            YAPnuc = results_cur[results_idx[2]]
            YAPratio = YAPnuc / (YAPphos + YAPnp)
            YAPTAZ_ratios[i][:] = YAPratio[-1]
            pFAK = results_cur[pFAK_idx]
            RhoA_GTP = results_cur[RhoA_idx]
            RhoA_GTP *= PM_area
            FActin = results_cur[FActin_idx]
            MyoA = results_cur[Myo_idx]
            pFAK_SS[i][:] = pFAK[-1]
            RhoA_GTP_SS[i][:] = RhoA_GTP[-1]
            FActin_SS[i][:] = FActin[-1]
            MyoA_SS[i][:] = MyoA[-1]
            if i == 10 and np.isclose(wasp_val, 0.01):
                plt.plot(results_cur[0], YAPratio,
                        label=f"h{heightArray[i]}_p{pitchArray[i]}_r{radiusArray[i]}_cellRad{cellRadArray[i]}")
        else:
            for j in range(len(curv0Array)):
                file_cur = (f"{cur_dir}/analysis_data/npy-files-nanopillars-withwasp/" 
                        f"nanopillars_h{heightArray[i]}_p{pitchArray[i]}_r{radiusArray[i]}_" 
                        f"cellRad{cellRadArray[i]}_nprate_curvSens{curv0Array[j]}_wasp{wasp_val}_results_all.npy")
                results_cur = np.load(file_cur)
                YAPphos = results_cur[results_idx[0]]
                YAPnp = results_cur[results_idx[1]]
                YAPnuc = results_cur[results_idx[2]]
                YAPratio = YAPnuc / (YAPphos + YAPnp)
                YAPTAZ_ratios[i][j] = YAPratio[-1]
                errors[i][j] = (YAPratio[-1] - yapData[i])**2 / yapSEMData[i]**2
                pFAK = results_cur[pFAK_idx]
                RhoA_GTP = results_cur[RhoA_idx]
                RhoA_GTP *= PM_area
                FActin = results_cur[FActin_idx]
                MyoA = results_cur[Myo_idx]
                pFAK_SS[i][j] = pFAK[-1] # SS FAK
                RhoA_GTP_SS[i][j] = RhoA_GTP[-1]
                FActin_SS[i][j] = FActin[-1]
                MyoA_SS[i][j] = MyoA[-1]
                if i in [1,7] and np.isclose(curv0Array[j], 5) and np.isclose(wasp_val, 0.01):
                    plt.plot(results_cur[0], YAPratio,
                        label=f"h{heightArray[i]}_p{pitchArray[i]}_r{radiusArray[i]}_cellRad{cellRadArray[i]}")
    errors_list.append(errors)

plt.legend()
plt.ylabel("YAP/TAZ N/C")
plt.xlabel('Time (s)')
plt.xlim([0, 3600])
# plt.savefig("cytosolic activation.pdf", format="pdf")

Plot summary of cytosolic stiffness, stress fibers, and pFAK at steady state (Fig 1C-F)

In [ ]:
vars = [9e-6*FActin_SS**2.6, FActin_SS*MyoA_SS/548.444, pFAK_SS]
names = ["Cyto stiffness", "Stress fibers", "pFAK"]
for i in range(len(vars)):
    plt.figure()
    cur_var = vars[i]
    name = names[i]
    width = 0.75  # the width of the bars

    x = [0,1,2]
    fig, ax = plt.subplots(layout='constrained', figsize=(3,3))
    ax.bar(x, cur_var[[10,1,7],3], width)
    ax.set_ylabel(name)
    ax.set_xticks(x, ['flat','100 nm', '250 nm'])
# plt.savefig("pFAK_bars.pdf", format="pdf")

Plot NSSE as a function of alpha0 (curvature sensitivity parameter):

In [ ]:
curvSens = [0, 1/10, 1/5, 1/2, 1]
errorsSel0 = [np.sum(errors_list[0][:,0]), np.sum(errors_list[0][:,4]), np.sum(errors_list[0][:,3]),
            np.sum(errors_list[0][:,2]), np.sum(errors_list[0][:,1])]
plt.plot(curvSens, errorsSel0, 'o-', label="kWASP=0.001")
errorsSel1 = [np.sum(errors_list[1][:,0]), np.sum(errors_list[1][:,4]), np.sum(errors_list[1][:,3]),
            np.sum(errors_list[1][:,2]), np.sum(errors_list[1][:,1])]
plt.plot(curvSens, errorsSel1, 'o--', label="kWASP=0.01")
# plt.ylim([100,300])
plt.ylabel("Normalized SSE")
plt.xlabel("Curvature sensitivity")
plt.legend()
# plt.savefig("curv_sens_fit.pdf", format="pdf")

Compare simulation YAP/TAZ N/C to experimentally measured values. If `multCond` is true, then plot for different levels of curvature sensitivity (Fig S1), otherwise create graph shown in Fig 2.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
multCond = False

conditions = []
for i in [10,0, 1, 2, 7, 6]:
    conditions.append(f"h{heightArray[i]}p{pitchArray[i]}r{radiusArray[i]}")
YAPTAZ_expt = np.array(yapData)
YAPTAZ_expt_std = np.array(yapStdData)
if multCond:
    np_means = {
        'Experiments': YAPTAZ_expt[[10,0,1,2,7,6]],
        'No curv sens': YAPTAZ_ratios[[10,0,1,2,7,6],0],
        'H0=10': YAPTAZ_ratios[[10,0,1,2,7,6],4],
        'H0=5': YAPTAZ_ratios[[10,0,1,2,7,6],3],
        'H0=2': YAPTAZ_ratios[[10,0,1,2,7,6], 2],
    }
    width = 0.12  # the width of the bars
else:
    np_means = {
        'Experiments': YAPTAZ_expt[[10,0,1,2,7,6]],
        'H0=5': YAPTAZ_ratios[[10,0,1,2,7,6],3],
    }
    width = 0.3

x = np.arange(len(conditions))  # the label locations
multiplier = 0

fig, ax = plt.subplots(layout='constrained', figsize=(6,4))

for attribute, measurement in np_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    # ax.bar_label(rects, padding=3)
    multiplier += 1

ax.errorbar(x, YAPTAZ_expt[[10,0,1,2,7,6]], YAPTAZ_expt_std[[10,0,1,2,7,6]], linestyle='None', color="r", capsize=4)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('YAP/TAZ N/C')
ax.set_xticks(x + width/2, conditions)
ax.legend()#loc='upper right')
plt.ylim([0.0, 4.5])
# plt.savefig("bargraph_multcond.pdf", format="pdf")

Plot NPC import rate as a function of stretch for different stretch sensitivites (Fig 4E).

In [ ]:
stretch_vals = np.linspace(0.1, 10, 200)
plt.loglog(stretch_vals, np.ones_like(stretch_vals),label='no stretch sens.')
plt.loglog(stretch_vals, np.exp((stretch_vals-1)/2.5),label='high stretch sens.')
plt.loglog(stretch_vals, np.exp((stretch_vals-1)/5.0), label='moderate stretch sens.')
plt.loglog(stretch_vals, np.exp((stretch_vals-1)/7.5), label='low stretch sens.')

plt.xlabel('Stretch ratio')
plt.ylabel('Relative import rate')
plt.legend()
# plt.savefig("/root/shared/gitrepos/smart-comp-sci-data/pyplots/nanopillars/stretch_transport.pdf", format="pdf")

Plot YAP/TAZ N/C for different stretch sensitivities, store SS YAP/TAZ N/C.

In [ ]:
indentationArray = [0.0, 0.4, 0.8, 
                    1.0, 1.4, 1.8, 
                    2.0, 2.4, 2.8]
a0Array=[0, 2.5, 5.0, 7.5]
var_names_saved = ["Cofilin_NP", "FActin", "GActin", "LaminA", "LIMK_A", "mDia_A", 
                   "Myo_A", "NPC_A", "pFAK", "RhoA_GDP", "RhoA_GTP", "ROCK_A", 
                   "Substrate_A", "YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_phos"]
plot_name = "FActin"
results_idx = var_names_saved.index(plot_name) + 1 # add one because time is first el
plot_names = ["YAPTAZ_phos", "YAPTAZ", "YAPTAZ_nuc"]
results_idx = []
for name in plot_names:
    results_idx.append(var_names_saved.index(name) + 1) # add one because time is first el

fig, ax = plt.subplots()

YAPTAZ_ratios = np.zeros([len(a0Array), len(indentationArray)])

for i in range(len(a0Array)):
    for j in range(len(indentationArray)):
        for domain in ["all"]:
            file_cur = (f"{cur_dir}/analysis_data/npy-files-nanopillars-withwasp-indent"
                    f"/nanopillars_indent{indentationArray[j]}_a0_{a0Array[i]}_results_{domain}.npy")
            try:
                results_cur = np.load(file_cur)
            except:
                continue
            if a0Array[i] > 0 and indentationArray[j] > 0: # then additional a0 variable included 
                YAPphos = results_cur[results_idx[0]+1]
                YAPnp = results_cur[results_idx[1]+1]
                YAPnuc = results_cur[results_idx[2]+1]
            else:
                YAPphos = results_cur[results_idx[0]]
                YAPnp = results_cur[results_idx[1]]
                YAPnuc = results_cur[results_idx[2]]
            YAPratio = YAPnuc / (YAPphos + YAPnp)
            YAPTAZ_ratios[i][j] = YAPratio[-1]
            if a0Array[i] == 5.0 and indentationArray[j] == 2.8:
                YAPTAZ_ratio_ref = YAPratio[-1]
                YAPTAZ_dyn_ref = [results_cur[0], YAPratio]
                NPC_ref = results_cur[8][-1]
                phiRef = YAPphos[-1]/(YAPphos[-1] + YAPnp[-1])
            if domain=="center_low":
                plt.plot(results_cur[0], YAPratio,
                    label=f"indent{indentationArray[j]}_a0_{a0Array[i]} low", linestyle="dashed")
            else:
                plt.plot(results_cur[0], YAPratio,
                label=f"indent{indentationArray[j]}_a0_{a0Array[i]}")
# plt.legend()
plt.ylabel("YAP/TAZ N/C")
plt.xlabel('Time (s)')
plt.legend()
plt.xlim([0, 3600])
# plt.ylim([2.9, 3.2])
# plt.savefig("YAPTAZ_withcompression.pdf", format="pdf")

Plot SS YAP/TAZ as a function of indentation for different stretch sensitivities.

In [ ]:
selIdx1 = np.array([0,1,3,4,5,6,7,8])
plt.plot(np.array(indentationArray), YAPTAZ_ratios[0], marker="o",label="no stretch sensitivity")
plt.plot(indentationArray, YAPTAZ_ratios[1], marker="o", label="with highly stretch sensitive NPCs")
plt.plot(indentationArray, YAPTAZ_ratios[2], marker="o", label="with moderately stretch sensitive NPCs")
plt.plot(indentationArray, YAPTAZ_ratios[3], marker="o", label="with low stretch sensitive NPCs")
# plt.ylim([2, 3])
# plt.xlim([-.05, 1.55])
plt.xlabel("Indentation (μm)")
plt.ylabel("YAP/TAZ N/C")
plt.legend()
# plt.savefig("indentation_NCsummary.pdf", format="pdf")

Create disk mesh to use for projection of F-actin and Lamin A steady state data.

In [ ]:
rad = 6.0
import gmsh
import numpy as np
import pathlib
from smart import mesh_tools
gmsh.initialize()
gmsh.model.add("arc")
surf = gmsh.model.occ.add_disk(0, 0, 0, rad, rad)
gmsh.model.occ.synchronize()
gmsh.model.add_physical_group(2, [surf], tag=1)
facets = gmsh.model.getBoundary([(2, surf)])
gmsh.model.add_physical_group(1, [facets[0][1]], tag=10)
def meshSizeCallback(dim, tag, x, y, z, lc):
    return 0.15

gmsh.model.mesh.setSizeCallback(meshSizeCallback)
# set off the other options for mesh size determination
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
# this changes the algorithm from Frontal-Delaunay to Delaunay,
# which may provide better results when there are larger gradients in mesh size
gmsh.option.setNumber("Mesh.Algorithm", 5)
gmsh.model.mesh.generate(3)
tmp_folder = pathlib.Path(f"tmp_ellipse")
tmp_folder.mkdir(exist_ok=True)
gmsh_file = tmp_folder / "ellipses.msh"
gmsh.write(str(gmsh_file))
gmsh.finalize()
dmesh, mf1, mf2 = mesh_tools.gmsh_to_dolfin(str(gmsh_file), tmp_folder, 2)
# remove tmp mesh and tmp folder
gmsh_file.unlink(missing_ok=False)
tmp_folder.rmdir()

Compute projection of FActin and LaminA at the z = 2 plane at steady-state. First need to compute the interpolation of 2.8 micron indentation by running interp_meshes.

In [ ]:
import sys
import numpy as np
import dolfin as d
from matplotlib import pyplot as plt
sys.path.append("/root/shared/gitrepos/smart-nanopillars/utils")
from smart_analysis import load_solution
mesh_parent_folder = "/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars_finalCalcFULL"
results_parent_folder = "/root/scratch/nanopillar-sims/results_nanopillars_indentation_REDO5sens"
tVec = np.loadtxt(f"{results_parent_folder}/nanopillars_indent2.8_a0_5.0/tvec.txt")
stress_fiber_int = []
NPC_int = []
idx = 500
for sol in load_solution(mesh_file=f"{mesh_parent_folder}/nanopillars_indent2.8_redo/spreadCell_mesh.h5", 
                            results_file=[f"{results_parent_folder}/interp/nanopillars_indent2.8_a0_5.0_interp/FActin_interp.h5", 
                                          f"{results_parent_folder}/interp/nanopillars_indent2.8_a0_5.0_interp/LaminA_interp.h5"],
                            idx=idx):
    FActin, LaminA = sol[:]
    dxCyto = d.Measure("dx", FActin.function_space().mesh())
    dxNE = d.Measure("dx", LaminA.function_space().mesh())
    CytoSubmesh = dmesh
    CytoCoordsRef = CytoSubmesh.coordinates()
    VFlat = d.FunctionSpace(CytoSubmesh, "P", 1)
    FActinProj = d.Function(VFlat)
    FActinProjVec = FActinProj.vector().get_local()
    LaminAProj = d.Function(VFlat)
    LaminAProjVec = FActinProj.vector().get_local()
    CytoCoords = VFlat.tabulate_dof_coordinates()
    multFactor = 1/(0.4*0.4*2*0.4*np.pi**(3/2))
    for i in range(len(CytoCoords)):
        curKernel = d.Expression(f"{multFactor}*exp(-(pow(x[0]-({CytoCoords[i,0]}),2) + "
                                 f"pow(x[1]-({CytoCoords[i,1]}),2) + pow((x[2]-2.0)/2,2))/0.16)", 
                                 degree=1)
        FActinProjVec[i] = d.assemble(curKernel*FActin*dxCyto)
        LaminAProjVec[i] = d.assemble(curKernel*LaminA*dxNE)
        print(f"{i+1} of {len(CytoCoords)}")
    FActinProj.vector().set_local(FActinProjVec)
    FActinProj.vector().apply("insert")
    LaminAProj.vector().set_local(LaminAProjVec)
    LaminAProj.vector().apply("insert")
    d.File("FActinProj.pvd") << FActinProj
    d.File("LaminAProj.pvd") << LaminAProj
    break

In [ ]:
stiffness_vec = [0.1, 5.7, 70000000.0]
radius=0.5
pitch=3.5
height=3.0
cellRad=15.5
EMod=10000000

indentation = 2.8#[0.8, 1.8, 2.8]
poreLocArray= [0.0, 0.0]#, 3.5, 3.5, 3.5]
poreSizeArray=[0.1, 0.2]#, 0.1, 0.2, 0.5]
poreRateArray=[1]
transportRateArray=[100, 10000]#, 100, 100, 10000, 10000, 10000]
transportRatioArray=[5, 5]#[1, 2, 5, 1, 2, 5]

var_names_saved = ["YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_nuc_phos", "YAPTAZ_phos"]
plot_names = ["YAPTAZ_phos", "YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_nuc_phos"]
results_idx = []
for name in plot_names:
    results_idx.append(var_names_saved.index(name) + 1) # add one because time is first el
fig, ax = plt.subplots()

pore_dir = f"{cur_dir}/analysis_data/npy-files-nanopillars-withwasp-pores"

YAPTAZ_ratios = np.zeros([len(poreSizeArray),len(poreRateArray),len(transportRateArray)])

for i in range(len(poreSizeArray)):
    for j in range(len(poreRateArray)):
        for k in range(len(transportRateArray)):
            loc = ["all"]
            for l in loc:
                cur_file = (f"nanopillars_indent{indentation}_pore_size{poreSizeArray[i]}_"
                        f"loc{poreLocArray[i]}_rate{poreRateArray[j]}_transport{transportRateArray[k]}_"
                        f"ratio{transportRatioArray[k]}_results_all.npy")
                cur_file = f"{pore_dir}/{cur_file}"
                try:
                    results_cur = np.load(cur_file)
                except:
                    continue
                if results_cur[0][-1] < 1e5:
                    print("short")
                YAPphos = results_cur[results_idx[0]]
                YAPnp = results_cur[results_idx[1]]
                YAPnuc = results_cur[results_idx[2]]
                YAPphos_nuc = results_cur[results_idx[3]]
                YAPratio = (YAPnuc + YAPphos_nuc) / (YAPphos + YAPnp)
                if l == "center_low":
                    # print("blank")
                    plt.plot(results_cur[0]/3600, YAPratio, label=f"Entry/exit ratio = {transportRatioArray[k]}", linestyle="dashed")
                elif l == "all":
                    plt.plot(results_cur[0]/3600, YAPratio, label=f"Pore size = {poreSizeArray[i]}")
                YAPTAZ_ratios[i][j][k] = YAPratio[-1]
tPlot = np.concatenate((YAPTAZ_dyn_ref[0]/3600, [3.0]))
YPlot = np.concatenate((YAPTAZ_dyn_ref[1], [YAPTAZ_dyn_ref[1][-1]]))
plt.plot(tPlot, YPlot, linestyle="dashed", label="no pore")
plt.legend()
plt.ylabel("YAP/TAZ N/C")
plt.xlabel('Time (h)')
# plt.title("Slow entry through pore")
plt.xlim([0, 3])
# plt.ylim([0, 7.5])
plt.savefig("YAPTAZ_poreDynamics_0.1pore.pdf", format="pdf")

In [ ]:
plt.plot(np.array(transportRatioArray[0:3]), YAPTAZ_ratios[1][0][0:3], marker="o",label="slow entry through pore")
plt.plot(np.array(transportRatioArray[3:]), YAPTAZ_ratios[1][0][3:], marker="o",label="fast entry through pore")
plt.plot(np.array(transportRatioArray[0:3]), YAPTAZ_ratio_ref*np.ones_like(YAPTAZ_ratios[2][1][0:3]), 
         linestyle="dashed",label="no pore")
# theoretical predictions for well mixed
ratios = np.linspace(1, 10, 100)
kin = 1.0 + 10.0*NPC_ref
kin = kin*1.825
kout = 1
kout_pore = 1000
kin_pore = kout_pore*ratios
SAnuc = 390
SApore = np.pi*.2**2
phiPhos = phiRef
# pred1 = ratios*(kin*(1+zeta1-phiRef) + kout*ratios*phiRef)/(zeta1*kin + kout*ratios)
correction = 1# - np.exp(-SApore*kin_pore*10000/(550*602.2))
pred1 = correction*ratios*phiPhos + (SAnuc*kin*(1-phiPhos) + SApore*kin_pore*(1-phiPhos))/(SAnuc*kout + SApore*kout_pore)
# pred1 = (SAnuc*kin*(1-phiPhos) + SApore*kin_pore*(1-phiPhos))/(SAnuc*kout + SApore*kout_pore)
plt.plot(ratios, pred1, label="Well-mixed prediction")
# plt.plot(np.array(transportRatioArray[0:3]), YAPTAZ_ratios[0][1][0:3], marker="o",label="transport = 10")
# plt.plot(np.array(transportRatioArray[3:]), YAPTAZ_ratios[0][1][3:], marker="o",label="transport = 100")
# plt.ylim([2, 3])
# plt.xlim([-.05, 1.55])
plt.xlabel("Transport ratio")
plt.ylabel("YAP/TAZ N/C")
plt.legend()

In [21]:
import dolfin as d
from smart import mesh_tools
import pathlib
import numpy as np
loaded = mesh_tools.load_mesh(pathlib.Path("/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars_finalCalcCoarse/nanopillars_indent2.8") / "spreadCell_mesh.h5")
mesh_cur = d.create_meshview(loaded.mf_facet, 12)
xCoords = mesh_cur.coordinates()[:,0].copy()
yCoords = mesh_cur.coordinates()[:,1].copy()
mesh_cur.coordinates()[:,0] = yCoords
mesh_cur.coordinates()[:,1] = xCoords
mesh_cur.coordinates()[:,0] *= -1
mesh_cur.coordinates()[:,1] *= -1
Vcur = d.FunctionSpace(mesh_cur, "P", 1)
xShift = 0.0
centerLogic = np.logical_and(np.isclose(mesh_cur.coordinates()[:,0], xShift, 0.1),
               np.logical_and(
               np.isclose(mesh_cur.coordinates()[:,1], 0.),
               mesh_cur.coordinates()[:,2] < 0.8*np.max(mesh_cur.coordinates()[:,2])))
centerIdx = np.nonzero(centerLogic)[0]
centerIdx = centerIdx[np.argmin(np.abs(mesh_cur.coordinates()[centerIdx,0]-xShift))]
zCenter = mesh_cur.coordinates()[centerIdx,2]
tVals = np.linspace(0., 5., 200)
pore_xdmf = d.XDMFFile(d.MPI.comm_world, "pore8.xdmf")
pore_xdmf.parameters["flush_output"] = True
t = d.Constant(tVals[0])
poreRad = 0.1
pore_expr = d.Expression(f"(1-exp(-tCur/1.0))*exp(-(pow(x[0]-{xShift},2)+pow(x[1],2)+pow(x[2]-{zCenter},2))/(pow(poreRad,2)))", tCur = t, degree=1, poreRad=poreRad)
pore_fcn = d.interpolate(pore_expr, Vcur)
for i in range(len(tVals)):
    t.assign(tVals[i])
    pore_fcn = d.interpolate(pore_expr, Vcur)
    pore_fcn.rename("pore", "pore")
    pore_xdmf.write(pore_fcn, tVals[i])

In [ ]:
import sys
import dolfin as d
sys.path.append("/root/shared/gitrepos/smart-nanopillars/utils")
import spread_cell_mesh_generation as mesh_gen
dmesh, mf2, mf3 = mesh_gen.create_substrate(nanopillars=[0.25, 1.0, 2.5], hEdge = 0.2, LBox=50, contact_rad=17.45)
# dmesh, mf2, mf3 = mesh_gen.create_substrate(nanopillars=[0.0, 0.0, 0.0], hEdge = 0.2, LBox=50)
d.ALE.move(dmesh, d.Expression(("0","0","-0.05"), degree=1))
d.File("substrate_250_omitNP.pvd") << dmesh

In [ ]:
from matplotlib import pyplot as plt
tVals = np.linspace(0., 50., 200)
poreVals = (1-np.exp(-tVals/10.0))
plt.xlabel("Time (s)")
plt.ylabel("permeability")
plt.plot(tVals, poreVals)

In [ ]:
moveNucArray = [0.0, -0.6, -1.0, -1.4]
var_names_saved = ["Cofilin_NP", "FActin", "GActin", "LaminA", "LIMK_A", "mDia_A", 
                   "Myo_A", "NPC_A", "pFAK", "RhoA_GDP", "RhoA_GTP", "ROCK_A", 
                   "Substrate_A", "YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_phos"]
plot_names = ["YAPTAZ_phos", "YAPTAZ", "YAPTAZ_nuc"]
results_idx = []
for name in plot_names:
    results_idx.append(var_names_saved.index(name) + 1) # add one because time is first el
fig, ax = plt.subplots()

YAPTAZ_ratios = np.zeros([len(moveNucArray), 1])

for i in range(len(moveNucArray)):
    file_cur = (f"{cur_dir}/analysis_data/npy-files-nanopillars-movenuc"
                    f"/nanopillars_movenuc{moveNucArray[i]}_results_all.npy")
    try:
        results_cur = np.load(file_cur)
    except:
        continue
    YAPphos = results_cur[results_idx[0]]
    YAPnp = results_cur[results_idx[1]]
    YAPnuc = results_cur[results_idx[2]]
    YAPratio = YAPnuc / (YAPphos + YAPnp)
    YAPTAZ_ratios[i] = YAPratio[-1]
    plt.plot(results_cur[0], YAPratio,
        label=f"movenuc{moveNucArray[i]}")
plt.ylabel("YAP/TAZ N/C")
plt.xlabel('Time (s)')
plt.legend()
plt.xlim([0, 3600])
# plt.ylim([2.9, 3.2])
plt.savefig("YAPTAZ_movenuc.pdf", format="pdf")